In [42]:
from pathlib import Path
from dataclasses import dataclass
import itertools

In [55]:
def get_input_data():
    input_file = Path(r"M:\Python\Projects\AdventOfCode\Day_4\Day_4_input.txt")
    return input_file.read_text().strip().split('\n')

test_input_data = [
        "7,4,9,5,11,17,23,2,0,14,21,24,10,16,13,6,15,25,12,22,18,20,8,19,3,26,1",
        "",
        "22 13 17 11  0",
        "8  2 23  4 24",
        "21  9 14 16  7",
        "6 10  3 18  5",
        "1 12 20 15 19",
        "",
        "3 15  0  2 22",
        "9 18 13 17  5",
        "19  8  7 25 23",
        "20 11 10 24  4",
        "14 21 16 12  6",
        "",
        "14 21 17 24  4",
        "10 16 15  9 19",
        "18  8 23 26 20",
        "22 11 13  6  5",
        "2  0 12  3  7",
    ]

@dataclass
class PointInformation:
    value: int
    is_checked: bool = False


def _check_if_won(points_info: list[list[PointInformation]]):
    return any(all((point_info.is_checked for point_info in row)) for row in points_info)


@dataclass
class Board:
    points: list[list[PointInformation]]
    used_numbers: set[int]

    def __contains__(self, item):
        return item in self.used_numbers

    def set_value_checked(self, value: int):
        for row in self.points:
            for point_info in row:
                if point_info.value == value:
                    point_info.is_checked = True

    def check_if_won(self):
        won_by_rows = self._won_by_rows()
        won_by_columns = self._won_by_columns()
        return won_by_rows or won_by_columns

    def _won_by_rows(self):
        return _check_if_won(self.points)

    def _won_by_columns(self):
        return _check_if_won(self._transposed_points())

    def _transposed_points(self):
        return [*zip(*self.points)]

    def sum_of_unmarked(self):
        calculated_sum = sum(
            point_info.value for point_info in itertools.chain.from_iterable(self.points) if not point_info.is_checked)
        return calculated_sum


def get_drawn_numbers(input_data):
    line_with_drawn_numbers = input_data[0]
    return [
        int(number) for number in line_with_drawn_numbers.split(",")
    ]


def get_boards(input_data):
    board_data = []
    board_number = 0
    current_points: list[list[PointInformation]] = []
    current_set_of_numbers = set()
    for line in input_data[2:]:
        if not line.strip():
            board_data.append(Board(current_points, current_set_of_numbers))
            current_points = []
            current_set_of_numbers = set()
            continue
        tmp = []
        for point_value in line.strip().split():
            point_value = int(point_value)
            tmp.append(PointInformation(value=point_value))
            current_set_of_numbers.add(point_value)
        current_points.append(tmp)
    board_data.append(Board(current_points, current_set_of_numbers))
    return board_data


def get_first_winning_board_score(input_data):
    drawn_numbers = get_drawn_numbers(input_data)
    boards = get_boards(input_data)
    for number in drawn_numbers:
        for idx, board in enumerate(boards):
            if number in board:
                board.set_value_checked(number)
                if board.check_if_won():
                    return board.sum_of_unmarked() * number


def test_bingo_board():

    winning_board_score = get_first_winning_board_score(test_input_data)
    expected_winning_score = 4512
    assert expected_winning_score == winning_board_score, f"{expected_winning_score =}, {winning_board_score =}"


test_bingo_board()

In [54]:
def calculate_winnings():
    input_data = get_input_data()
    print(get_first_winning_board_score(input_data))
calculate_winnings()

72770


In [63]:
def get_last_winning_board_score(input_data):
    drawn_numbers = get_drawn_numbers(input_data)
    boards = get_boards(input_data)
    for number in drawn_numbers:
        for idx, board in enumerate(boards):
            if number in board:
                board.set_value_checked(number)
                other_boards_won = all(other_boards.check_if_won() for other_boards in filter(lambda x: x != board, boards))
                if board.check_if_won() and other_boards_won:
                    return board.sum_of_unmarked() * number

def test_calculate_last_winnig_board_score():
    expected_score = 1924
    score = get_last_winning_board_score(test_input_data)
    assert score == expected_score, f"{score = }, {expected_score = }"

test_calculate_last_winnig_board_score()
def calculate_last_winnings():
    input_data = get_input_data()
    print(get_last_winning_board_score(input_data))
calculate_last_winnings()

13912
